# Εξόρυξη Δεδομένων
### Εργασία Χειμερινού Εξαμήνου 
### Ακαδημαικό Έτος 2019-2020
#### Στρίκλαντ Ελένη 3140252
#### Μπούσουλας-Ραϊκίδης Ορφέας-Γεώργιος 3160111
Όνομα ομάδας στο Kaggle: InfinityLoops w/o Neural Networks

## Εισαγωγή 

Στο πλάισιο του μαθήματος Εξόρυξη Δεδομένων απο Βάσεις Δεδομένων και τον Παγκόσμιο Ιστό μας ανατέθηκε να λύσουμε ένα πρόβλημα παλινδρόμησης. Συγκεκριμένα, μας δόθηκε ένα σύνολο δεδομένων που αποτελείται από μερικές χιλιάδες εγγραφές που περιέχουν την ωριαία μέτρηση των ενοικιαζόμενων ποδηλάτων μεταξύ των ετών 2011 και 2012 στο σύστημα Capital bikeshare με τις αντίστοιχες καιρικές και εποχιακές πληροφορίες.

Ο στόχος μας είναι να προβλέψουμε πόσα ποδήλατα θα ενοικιάζονται κάθε ώρα της ημέρας, με βάση δεδομένα όπως ο καιρός, ο χρόνος, η θερμοκρασία, αν η ημέρα είναι εργασιμή ή οχι, ή έαν είναι περιοδος διακοπών ή εορτών.

In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import stats
from numpy import median
from sklearn import metrics
from sklearn import model_selection
from sklearn.svm import LinearSVR
from sklearn.datasets import make_regression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_log_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
#RMSLE score calculation
def rmsle_score(y_true, y_pred):
    for i, y in enumerate(y_pred):
        if y_pred[i] < 0:
            y_pred[i] = 0
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [4]:
#Reading Datasets
filename = 'dataset/train.csv' 
df_train = pd.read_csv(filename)

df_train.shape
df_test = pd.read_csv('dataset/test.csv')

df_test.shape
df_train.isnull().sum()
df_train.rename(columns={'weathersit':'weather',
                     'mnth':'month',
                     'hr':'hour',
                     'yr':'year',
                     'hum': 'humidity',
                     'cnt':'count'},inplace=True)

#Not used in final submission
# df_train.dtypes
# df_train['season'] = df_train.season.astype('category')
# df_train['year'] = df_train.year.astype('category')
# df_train['month'] = df_train.month.astype('category')
# df_train['weekday'] = df_train.weekday.astype('category')
# df_train['hour'] = df_train.hour.astype('category')
# df_train['holiday'] = df_train.holiday.astype('category')
# df_train['workingday'] = df_train.workingday.astype('category')
# df_train['weather'] = df_train.weather.astype('category')
# df_train.dtypes

#Setting training data
df_train = df_train.drop(['windspeed','atemp', 'casual', 'registered'], axis=1)
X = df_train[['temp', 'humidity', 'hour', 'month','workingday', 'holiday', 'season','weekday', 'year', 'weather']]
y = df_train['count']
# Training and test data is created by splitting the main data. 20% of test data is considered
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Ενότητα 1: Ανάλυση Δεδομένων

Πριν ξεκινήσουμε να δοκιμάζουμε αλγορίθμους machine learning προσπαθήσαμε να δουλέψουμε λίγο με τα δεδομένα που μας δόθηκαν. Παρατηρήσαμε ότι πολλά απο τα attributes είναι κατηγορικές μεταβλητές και δουλέψαμε λίγο με τον κώδικα που μας δόθηκε απο την εκφώνηση ο οποίος σε πρώτο στάδιο χρησιμοποιεί τον Linear Regression. 

#### Μέγεθος Test dataset

Δοκιμάσαμε αρχικά να αλλάξουμε το μέγεθος του test dataset και αντί για 30% να παίρνει περισσότερα δεδομένα από το train dataset, πράγμα που οδήγησε σε μία ελαφρά μείωση του σκορ διατηρώντας ίδια χαρακτηριστικά.  Ωστόσο, αποφασίσουμε να κρατήσουμε το 20%, έτσι ώστε να βελτιώσουμε πρώτα τον αλγόριθμο χωρίς να αυξηθεί ο χρόνος εκτέλεσης του.

#### Χρονική Ανάλυση
Η αμέσως επόμενη προσσέγγιση του προβλήματος ήταν η χρονική ανάλυση. Για το σκοπό αυτό δημιουργήσαμε το χαρακτηριστικό Date που αποτελεί μία συννένωση του year και month. Με τη βοήθεια της pandas μετατρέψαμε το year από την μορφή 0-1 σε 2011-2012 (που είναι τα έτη των μετρήσεων) και παράξαμε στοιχεία της μορφής year-month (π.χ. 2011-01) ως datetime64. Έπειτα δημιουργήσαμε ένα διάγραμμα για να δούμε την εξέλιξη του count σε συνάρτηση με τη ροή του χρόνου (2011-01  έως 2012-12). 

![Εικόνα διαγράμματος χρόνου](img/timeFlowWanted.png)

Αυτό που παρατηρήσαμε από το συγκεκριμένο διάγραμμα είναι πως το 2012 νοικιάστηκαν περισσότερα ποδήλατα από ότι το 2011 στους ίδιους μήνες, ενώ σημαντικό ρόλο διαδραματίζει η εποχή καθώς τους καλοκαιρινούς μήνες νοικιάζονται πολλά παραπάνω ποδήλατα από ότι τους χειμερινούς. Ωστόσο, δεν χρησιμοποιήσαμε αυτή την προσσέγγιση τελικά.

In [7]:
# df_train['year']=df_train['year'].apply(lambda x: '2011' if x == '0' else '2012')
# df_train['date']=df_train[['year','month']].apply(lambda x: '-'.join(x), axis = 1)
# df_train['date']=pd.to_datetime(df_train['date'], errors='ignore')
# df_train=df_train.sort_values(by=['date', 'hour'])
# print(df_train)
# df_train['date'] = df_train.date.astype('datetime64').astype(int).astype(float)

# fig,[ax1,ax2] = plt.subplots(nrows=2, figsize=(20,10))
# sn_plot =sn.lineplot(x="date", y="count", data=df_train, palette="tab10", linewidth=2.5,ax = ax1)
# sn_plot.figure.savefig("timeFlow.png")

#### Φιλτράρισμα τιμών
Άλλη μία ιδέα που υλοποιήσαμε για να έχουμε καλύτερα αποτελέσματα είναι να χρησιμοποιήσουμε Outliers. Ουσιαστικά φτιάξαμε ένα διάγραμμα το οποίο έδειχνε για όλες τις εγγραφές πόσα ποδήλατα ενοικιάστηκαν. Παρατηρήσαμε ότι ο μέσος όρος κυμένεται απο το 300 έως το 500 οπότε αποφασίσαμε να αφαιρέσουμε απο το dataset όλες τις εγγραφές με count μεγαλύτερο του 500. Παρακάτω φαίνεται το αντίστοιχο διαγραμμα. 

![](img/outliers.png)

Αν και στον αλγόριμο Linear Regression η αφαίρεση των outliers έδινε καλύτερα αποτελέσματα, στους υπόλοιπους αλγορίθμους που βασίζονται κυρίως σε δέντρα τα αποτελέσματα ήταν λιγότερα, οπότε δεν χρησιμοποιήσαμε αυτή τη μέθοδο τελικά.

In [9]:
# df_train = df_train.drop(df_train[df_train['count'] > 500].index)
# df_train= df_train.sort_values(by='count')
# outliers_dataset=df_train['count'].copy()
# sn.relplot(data=outliers_dataset, ax=ax1)

## Ενότητα 2. Regressors

Να σημειωθεί ότι για την εύρεση καλύτερων αποτελεσμάτων χρησιμοποιούμε Grid Search CV σε όλους τους αλγορίθμους που θα εφαρμόσουμε στην συνέχεια, για την καλύτερη παραμετροποίηση τους. 

### Ενότητα 2.1: Linear Regression

Για την αρχική προσέγγιση του προβλήματος επικεντρωθήκαμε στο να αξιοποιήσουμε τον κώδικα που μας δόθηκε ως παράδειγμα από το εργαστήριο. Χρησιμοποιώντας, λοιπόν, τον αλγόριθμο Linear Regression και εκτελώντας τον κώδικα της εργασίας το score που υπολογίστηκε με την μέθοδο RMSLE (Root Mean Squared Logarithmic Error) και τα χαρακτιριστικά temp, humidity, workingday έδωσε 1.4089. 

##### Διαλέγοντας Χαρακτηριστικά
Επιλέγοντας διαφορετικά χαρακτηριστικά προσπαθήσαμε να δούμε πως κινείται το σκορ  και στον παρακάτω πίνακα φαίνονται τα αποτελέσματα για μερικούς συνδυασμούς με το αντίστοιχο score στα training data.

|χαρακτιριστικά|σκορ|
|----------------|--------|
|temp, year, month, weekday, hour, weather|**1.3129**|
|temp, month, hour, weather, season|**1.2795**|
|temp, month, hour, weather|**1.2867**|
|temp, hour, weather,season,weekday|**1.2829**|
|temp, hour, humidity|**1.2523**| 

Ακόμα, δοκιμάσαμε να χρησιμοποιήσουμε όλα τα χαρακτηριστικά (και τα 12), αλλά το σκορ ανέβαινε με κάθε επιπλέον χαρακτιριστικό που προσθέταμε.Οπότε αποφασίσαμε να χρησιμοποιήσουμε όσο το δυνατόν λιγότερα χαρακτηριστικά, για αυτό επιλέξαμε αυτά που μείωναν το σκορ περισσότερο στις δοκιμές που κάναμε στο τοπικό σετ δεδομένων. Έτσι καταλήξαμε σε:

```temp, humidity, hour με σκορ 1.2523```

Ωστόσο υποβάλλοντας τα αποτελέσματα στο Kaggle.com λάβαμε σκορ 1.64, οπότε καταλάβαμε πως αυτή η προσσέγγιση του προβλήματος δεν μας δίνει καλή γενίκευση, αν και δουλέυει καλά για το μικρό σετ δεδομένων που έχουμε.

In [9]:
# lr = LinearRegression(n_jobs = -1)
# parameters = {'normalize': [False]}
# lr_cv = GridSearchCV(lr, parameters, cv=5, n_jobs=-1, verbose = 1)
# lr_cv.fit(X_train, y_train)

# y_pred_lr = lr_cv.predict(X_test)

# print('Random Forest Best Parameters:', lr_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_lr))

### Ενότητα 2.2: Support Vector Machine

Ενώ η μέθοδος SVR είναι πολύ δημοφιλής για την λύση προβλημάτων παλινδρόμησης δεν μας βοήθησε ιδιαίτερα. Τα αποτελέσματα ήταν παρόμοια με την Linear Regression, ενώ εδινε λίγο καλύτερο score στα training data, 1,15 για την ακρίβεια, στο συνολικό dataset το score ανέβηκε στο 1,75. Η μέθοδος θα λέγαμε δεν είναι και πολύ κατάλληλη εφόσον ψάχνουμε μία εκτίμηση ως προς την μεταβλητή count, δηλαδή τον αριθμό των ποδηλάτων των οποίων ενοικιάστηκαν.


In [8]:
# svmr = LinearSVR(random_state=0)
# parameters = {'tol': [1e-03], 'max_iter': [10]}
# svmr_cv = GridSearchCV(svmr, parameters, cv=5, n_jobs=-1, verbose = 1)
# svmr_cv.fit(X_train, y_train)

# y_pred_svmr = svmr_cv.predict(X_test)

# print('SVM Parameters:', svmr_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_svmr))

### Ενότητα 2.3: Random Forest Regressor

Επειδή το dataset περιέχει πολλές κατηγορικές μεταβλητές αποφασίσαμε να να χρησιμοποιήσουμε των αλγόριθμο Random Forest Regressor, δεδομένου ότι οι αλγόριθμοι που χρησιμοποιούν δέντρα απόφασης είναι πολύ πιο αποδοτικά στο ζήτημα των κατηγορικών μεταβλητών. Τα αποτελέσματα στον σκορ ήταν ιδιαίτερα θετικά , αφου καταφέραμε να το ρίξουμε στο 0,33, οπότε αυτός θα είναι και ο βασικός αλγοριθμος που θα χρησιμοποιήσουμε. 

In [7]:
# clf = RandomForestRegressor(n_jobs = 4, random_state = 0, verbose = 1)
# parameters = {'n_estimators': [1000], 'max_depth': [25]}
# clf_cv = GridSearchCV(clf, parameters, cv=5, n_jobs=-1, verbose = 1)
# clf_cv.fit(X_train, y_train)

# y_pred = clf_cv.predict(X_test)

# print('Random Forest Best Parameters:', clf_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred))

### Ενότητα 2.4: Κ-Nearest Neighbors 

Επόμενος αλγόριθμος που επιλέξαμε να δοκιμάσουμε για τη λύση του προβλήματος παλινδρόμησης έιναι ο Knn. Ο Knn μας έδωσε score 0,39. Πολύ καλό αποτέλεσμα αλλα όχι κάτι καλύτερο απο τον Random Forest. 

In [6]:
# kn = KNeighborsRegressor(n_jobs = 4)
# parameters = {'n_neighbors': [7],  'weights': ['distance'], 'p': [1]}
# kn_cv = GridSearchCV(kn, parameters, cv=5, n_jobs=-1, verbose = 1)
# kn_cv.fit(X_train, y_train)

# y_pred_kn = kn_cv.predict(X_test)

# print('KNeighbors Parameters:', kn_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_kn))

### Ενότητα 2.5: Gradient Boosting Regressor

Ο Gradient Boosting Regressor είναι ένας καλός αλγόριθμος για την λύση του προβλήματος μας. Αν και είχε καλή απόδοση ήταν πάρα πολύ αργός (λόγο του ότι δεν τρέχει σε παραλληλία) γι αυτό και ορίσαμε 100 estimators με σκοπό να μειώσουμε την χρονική διάρκεια του. Τo score του στα training data είναι 0.37.

In [5]:
# gb = GradientBoostingRegressor(random_state = 42, verbose= 1)
# parameters = {'n_estimators': [100],  'criterion': ['mae'], 'max_depth': [15]}
# gb_cv = GridSearchCV(gb, parameters, cv=5, n_jobs=-1, verbose = 1)
# gb_cv.fit(X_train, y_train)

# y_pred_gb = gb_cv.predict(X_test)

# print('GradientBoosting Parameters:', gb_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_gb)) 

### Ενότητα 2.6: Extra Trees Regressor

Εφόσον ο Random Forest Regressor μας έριξε πάρα πολύ το score αποφασίσαμε να δοκίμασου παρόμοιο αλγόριθμο με decision trees, ο οποίος έδωσε το ίδιο score με τον Random Forest, 0,33.

In [4]:
# ext = ExtraTreesRegressor(random_state = 42, n_jobs= 4, verbose= 1)
# parameters = {'n_estimators': [100],  'criterion': ['mse'], 'max_depth': [25]}
# ext_cv = GridSearchCV(ext, parameters, cv=5, n_jobs=-1, verbose = 1)
# ext_cv.fit(X_train, y_train)

# y_pred_ext = ext_cv.predict(X_test)

# print('ExtraTree Parameters:', ext_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_ext)) 

### Ενότητα 2.7: Bagging Regressor

Το ίδιο score έδωσε και ο Bagging Regressor, 0,33.

In [3]:
# bag = BaggingRegressor(random_state = 42, verbose= 1)
# parameters = {'n_estimators': [5,10,50,100,1000]}
# bag_cv = GridSearchCV(bag, parameters, cv=5, n_jobs=-1, verbose = 1)
# bag_cv.fit(X_train, y_train)
# y_pred_bag = bag_cv.predict(X_test)

# print('Bagging Parameters:', bag_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_bag)) 

### Ενότητα 2.8: Hist Gradient Boosting Regressor

Ακόμα δοκιμάσαμε και έναν πειραματικό αλγόριθμο του sklearn, τον Hist Gradient Boosting Regressor. To score ήταν ικανοποιητικό στο 0.46 .

In [17]:
# hgb = HistGradientBoostingRegressor(random_state = 42)
# parameters = {'max_iter': [1000], 'max_depth':[20], 'learning_rate':[0.1]}
# hgb_cv = GridSearchCV(hgb, parameters, cv=5, n_jobs=-1)
# hgb_cv.fit(X_train, y_train)
# y_pred_hgb = hgb_cv.predict(X_test)

# print('HistGradientBoostingRegressor Parameters:', hgb_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_hgb))

HistGradientBoostingRegressor Parameters: {'learning_rate': 0.1, 'max_depth': 20, 'max_iter': 1000}
RMSLE: 0.4618802059113205


**Σημείωση: Δοκιμάσαμε και πολλούς άλλους αλγορίθμους, αλλά οι προσεγγίσεις ή τα αποτελέσματα τους δεν μας ικανοποίησαν και τόσο, οπότε δεν τους αναφέρουμε εδώ. Μερικά παραδείγματα Adaboost, DecisionTreeRegressor, KDTree.**

## Ενότητα 3: XGBoost 
Εκτός από τους αλγορίθμους του sklearn υλοποιήσαμε και μία εκδοχή του αλγορίθμου XGBRegressor από το πακέτο XGBoost. Από τα αποτελέσματα φάνηκε πως είναι ένας πολύ καλός αλγόριθμος που μας έδωσε score 0.2866. Οπότε αποφασίσαμε πως είναι ένας από τους βασικούς που θέλουμε να χρησιμοποιήσουμε.
Ωστόσο, για να μας δώσει αυτό το score κάναμε κάποιες αλλαγές στα δεδομένα μας. Συγμεκριμένα, δεν μετατρέψαμε κανένα από τα attributes σε κατηγορικό. Επίσης, κατά το fitting του μοντέλου χρησιμοποιούμε μία λογαριθμική συνάρτηση για το y_train. Στην συνέχεια επαναφέρουμε τα δεδομένα που προκύπτουν από το prediction του xgboost στην αρχική τους κατάσταση με μία εκθετική συνάρτηση.

In [2]:
# xg = XGBRegressor(subsample = 0.5, max_depth= 9, nthread = -1, silent = 1)

# xg.fit(X_train, np.log(y_train))
# y_pred = np.exp(xg.predict(X_test))
# print('RMSLE:', rmsle_score(y_test, y_pred))

## Ενότητα 4: Νευρωνικά Δίκτυα - SciKit Learn
Γενικά είπαμε να μην χρησιμοποιήσουμε νευρωνικά δίκτυα, κυρίως γιατί θέλαμε να πειραματιστούμε με μία λύση που θα ακολουθούσε μια διαφορετική προσσέγγιση. Όμως, κάναμε μία υλοποίηση του MLPRegressor που παρέχει το sklearn. Πειραματιζόμενοι με αυτή την μέθοδο καταφέραμε ένα score της τάξεως του 0.34. Βρήκαμε το συγκεκριμένο μοντέλο αργό και για αυτό δεν συνεχίσαμε παραπάνω, ενώ δεν καταφέραμε να το παραμετροποιήσουμε ώστε να μας δώσει καλύτερα αποτελέσματα από το μοντέλο του xgboost. 

In [1]:
# nn_cv = MLPRegressor(hidden_layer_sizes = (256,128,64,32,16), activation = 'relu', solver = 'adam', max_iter = 1000,learning_rate_init = 0.001, epsilon = 1e-8, random_state = 42, verbose = 1)
# parameters = {'epsilon':[1e-7] }
# nn_cv = GridSearchCV(nn_cv, parameters, cv=5, n_jobs=-1)
# nn_cv.fit(X_train, y_train)
# y_pred_nn = nn_cv.predict(X_test)

# print('MLPRegressor Parameters:', nn_cv.best_params_)
# print('RMSLE:', rmsle_score(y_test, y_pred_nn)) 

## Ενότητα 5: Συνδυασμός Μεθόδων (Voting) Τελική Υποβολή

Για την τελική μας λύση στο πρόβλημα, στραφήκαμε προς το μοντέλο του Voting. Δηλαδή των συνδυασμό πολλών αλγορίθμων που γίνονται fit μέσω του VotingRegressor του sklearn.

Οι αλγόριθμοι που επιλέξαμε είναι ο RandomForestRegressor, KNeighborsRegressor, ExtraTreesRegressor, BaggingRegressor, HistGradientBoostingRegressor και φυσικά ο XGBRegressor. Όπως και στον xgboost, λογαριθμούμε κατά το fitting και μετά επαναφέρουμε κατά το prediction τα δεδομένα μας. Το score στα training data είναι 0.28151 νώ η πλατφόρμα Kaggle έβγαλε 0.28634, το καλύτερο score μεχρι στιγμής. Για την υλοποίηση του Voting δίνουμε περισσότερο βάρος σε κάποιους αλγορίθμους:

|Αλγόριθμος|Βάρος|
|----------------|--------|
|RandomForestRegressor|1|
|KNeighborsRegressor|1|
|BaggingRegressor|1|
|HistGradientBoostingRegressor|1|
|ExtraTreesRegressor|2
|XGBRegressor|5

Η συγκεκριμένη μέθοδος βοηθά στο να βελτιώσουμε το αποτέλεσμα που προκύπτει από δεδομένα που δυσκολέυουν έναν αλγόριθμό, αλλά είναι πιο εύκολα διαχειρίσιμα από κάποιον άλλο.

In [23]:
clf = RandomForestRegressor(n_estimators=1000, max_depth=25, n_jobs = -1, random_state = 0)
kn = KNeighborsRegressor(n_neighbors=7,  weights='distance', p=1, n_jobs = -1)
ext = ExtraTreesRegressor(n_estimators=100,criterion='mse',max_depth=25, random_state = 42, n_jobs= -1)
bag = BaggingRegressor(n_estimators = 1000, random_state = 42)
hgb = HistGradientBoostingRegressor(max_iter = 1000, max_depth = 20, learning_rate = 0.1, random_state = 42)
xg = XGBRegressor(subsample = 0.5, max_depth= 9, nthread = -1, silent = 1)

voting = VotingRegressor(estimators=[('kn', kn), ('clf', clf), ('ext',ext), ('bag',bag), ('hgb', hgb),('xg',xg)], weights=[1,1,2,1,1,5], n_jobs=-1)
voting.fit(X_train,np.log(y_train))
y_pred_voting = np.exp(voting.predict(X_test))
print('Voting RMSLE score:', rmsle_score(y_test, y_pred_voting))

Voting RMSLE score: 0.2815137572214321


### Submission 
Τέλος, παράγουμε το αρχείο υποβολής για το Kaggle.

In [ ]:
filename = 'dataset/test.csv' 
df_test = pd.read_csv(filename)

df_test.shape
df_test.rename(columns={'weathersit':'weather',
                     'mnth':'month',
                     'hr':'hour',
                     'yr':'year',
                     'hum': 'humidity',
                     'cnt':'count'},inplace=True)

# =============================================================================
# df_test['season'] = df_test.season.astype('category')
# df_test['year'] = df_test.year.astype('category')
# df_test['month'] = df_test.month.astype('category')
# df_test['hour'] = df_test.hour.astype('category')
# df_test['holiday'] = df_test.holiday.astype('category')
# df_test['weekday'] = df_test.weekday.astype('category')
# df_test['workingday'] = df_test.workingday.astype('category')
# df_test['weather'] = df_test.weather.astype('category')
# =============================================================================

df_test = df_test.drop(['windspeed','atemp'], axis=1)
df_test = df_test[['temp', 'humidity', 'hour', 'month', 'workingday', 'holiday', 'season','weekday', 'year', 'weather']]
df_test.shape

y_pred = np.exp(voting.predict(df_test))
True in (y_pred < 0)
for i, y  in enumerate(y_pred):
    if y_pred[i] < 0:
        y_pred[i] = 0

submission = pd.DataFrame()
submission['Id'] = range(y_pred.shape[0])
submission['Predicted'] = y_pred
submission.to_csv("new_submission.csv", index=False)

Υ.Γ. Παραπάνω έχουμε βάλει αρκετά κομμάτια κώδικα σε σχόλια.
Για την εκτέλεση του κώδικα που είναι σε σχόλια:
1. Επιλέγουμε τον κώδικα που θέλουμε να τρέξουμε
2. Πατάμε ctrl+/ για να αφαιρεθεί από σχόλιο
3. Τρέχουμε το αντίστοιχο πεδίο του Notebook.

Για ευκολία παρέχουμε ένα ξεχωριστό αρχείο python που περιέχει όλο των κώδικα από τον οποίο προκύπτει το υποβληθέν αρχείο, χωρίς τα περιττά σχόλια που αφορούν μεθόδους που τελικά δεν ακολουθήσαμε, αλλά θέλαμε να αναφέρουμε.